#  Redes neuronales recurrentes
[**Python Deep Learning** Introducción práctica con Keras y TensorFlow 2. Jordi Torres. Editorial Marcombo ISBN: 9788426728289 ](https://www.marcombo.com/python-deep-learning-9788426728289/)

[**Redes neuronales recurrentes** Jordi Torres](https://torres.ai/redes-neuronales-recurrentes/)

# Caso de estudio: generacion de texto

Este caso de estudio trata de generar texto usando una RNN basada en caracteres. Se entrena un modelo de red neuronal para predecir el siguiente caracter a partir de una secuencia de caracteres. Con este modelo intencionadamente simple, se consigue generar secuencias de texto mas largas llamando al modelo repetivamente.


### Importar TensorFlow 2.0  y otras librerias

In [1]:
#tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import time

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


### Descarga del conjunto de datos

In [2]:

#from google.colab import files
#se debe cargar el fichero “Libro-Deep-Learning-introduccion-practica-con-Keras-1a-parte.txt”
#files.upload()

#path_to_fileDL ='/content/Libro-Deep-Learning-introduccion-practica-con-Keras-1a-parte.txt'

path_to_fileDL = tf.keras.utils.get_file('Shakespear.txt', 'https://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt')


In [3]:


text = open(path_to_fileDL, 'rb').read().decode(encoding='utf-8')
print('Longitud del texto:        {} carácteres'.format(len(text)))

vocab = sorted(set(text))

print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab)

Longitud del texto:        99993 carácteres
El texto está compuesto de estos 62 carácteres:
['\n', ' ', '!', "'", ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


Como se esta tratando el caso de estudio a nivel de caracter, podriamos considerar que aqui el corpus son los caracteres, por tanto un corpus muy pequeño.

Las redes neuronales solo procesan valores numericos, no letras, por tanto tenemos que traducir los caracteres a representacion numerica. Para ello se crean dos tablas de traduccion: una de caracteres a numeros y otra de numeros a caracteres:

In [4]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  ' ' :   1,
  '!' :   2,
  "'" :   3,
  ',' :   4,
  '-' :   5,
  '.' :   6,
  ':' :   7,
  ';' :   8,
  '?' :   9,
  'A' :  10,
  'B' :  11,
  'C' :  12,
  'D' :  13,
  'E' :  14,
  'F' :  15,
  'G' :  16,
  'H' :  17,
  'I' :  18,
  'J' :  19,
  'K' :  20,
  'L' :  21,
  'M' :  22,
  'N' :  23,
  'O' :  24,
  'P' :  25,
  'Q' :  26,
  'R' :  27,
  'S' :  28,
  'T' :  29,
  'U' :  30,
  'V' :  31,
  'W' :  32,
  'X' :  33,
  'Y' :  34,
  'Z' :  35,
  'a' :  36,
  'b' :  37,
  'c' :  38,
  'd' :  39,
  'e' :  40,
  'f' :  41,
  'g' :  42,
  'h' :  43,
  'i' :  44,
  'j' :  45,
  'k' :  46,
  'l' :  47,
  'm' :  48,
  'n' :  49,
  'o' :  50,
  'p' :  51,
  'q' :  52,
  'r' :  53,
  's' :  54,
  't' :  55,
  'u' :  56,
  'v' :  57,
  'w' :  58,
  'x' :  59,
  'y' :  60,
  'z' :  61,


Ahora tenemos una representacion de entero (integer) para cada caracter que podemos ver ejecutando el codigo previo.

Con esta funcion inversa a la anterior, podemos pasar el texto a enteros:

In [5]:
text_as_int = np.array([char2idx[c] for c in text])

Para comprobarlo podemos mostrar los 50 primeros caracteres del texto contenido en el tensor text_as_int:

In [6]:
print ('texto: {}'.format(repr(text[:50])))
print ('{}'.format(repr(text_as_int[:50])))

texto: "That, poor contempt, or claim'd thou slept so fait"
array([29, 43, 36, 55,  4,  1, 51, 50, 50, 53,  1, 38, 50, 49, 55, 40, 48,
       51, 55,  4,  1, 50, 53,  1, 38, 47, 36, 44, 48,  3, 39,  1, 55, 43,
       50, 56,  1, 54, 47, 40, 51, 55,  1, 54, 50,  1, 41, 36, 44, 55])


### Preparar los datos para entrenar la RNN


Para entrenar el modelo prepararemos unas secuencias de caracteres como entrada y salida de un tamaño determinado. En nuestro ejemplo se definio el tamaño de 100 caracteres con la variable seq_lenght.

Empezamos dividiendo el texto que tenemos en secuencias de caracteres con las cuales luego construiremos los datos de entrenamiento compuestos por las entradas de seq_lenght caracteres y las salidas correspondientes que cntienen la misma longitud de textom excepto que se desplaza un caracter a la derecha. 
Por ejemplo, suponiendo un seq_lenght=3 y teniendo como texto un "Hola", la secuencia de entrada seria "Hol", y la de salida "ola".

Se utilizara la siguiente funcion, que crea un conjunto de datos con el contenido del tensor text_as_int que contiene el texto, al que podremos aplicar el metodo batch() para dividir este conjunto de datos en secuencias de seq_lenght+1 de indice de caracteres.

In [7]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

seq_length = 100
 
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)



Podemos comprobar que sequences contiene el texto divido en paquetes de 101 caracteres como esperamos.

In [8]:
for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

"That, poor contempt, or claim'd thou slept so faithful,\nI may contrive our father; and, in their defe"
'ated queen,\nHer flesh broke me and puttance of expedition house,\nAnd in that same that ever I lament '
'this stomach,\nAnd he, nor Butly and my fury, knowing everything\nGrew daily ever, his great strength a'
"nd thought\nThe bright buds of mine own.\n\nBIONDELLO:\nMarry, that it may not pray their patience.'\n\nKIN"
'G LEAR:\nThe instant common maid, as we may less be\na brave gentleman and joiner: he that finds us wit'
"h wax\nAnd owe so full of presence and our fooder at our\nstaves. It is remorsed the bridal's man his g"
'race\nfor every business in my tongue, but I was thinking\nthat he contends, he hath respected thee.\n\nB'
"IRON:\nShe left thee on, I'll die to blessed and most reasonable\nNature in this honour, and her bosom "
'is safe, some\nothers from his speedy-birth, a bill and as\nForestem with Richard in your heart\nBe ques'
"tion'd on, nor that I was enough:\nWhic

De esta secuencia se obtiene el conjunto de datos de training que contenga tanto los datos de entrada (desde la posicion 0 a 99) como los datos de salida (desde la posicion 1 a la 100). Para ello se crea una funcion que realiza esta tarea y se aplica a todas las secuencias usando el metodo map() de la siguiente forma:

In [9]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


En este punto, dataset contiene un conjunto de parejas de secuencias de texto (con la representación numérica de los caracteres), donde el primer componente de la pareja contiene un paquete con una secuencia de 100 caracteres del texto original y la segunda su correspondiente salida, también de 100 caracteres. Podemos comprobarlo visualizándolo por pantalla (por ejemplo mostrando la primera pareja):

In [10]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "That, poor contempt, or claim'd thou slept so faithful,\nI may contrive our father; and, in their def"
Target data: "hat, poor contempt, or claim'd thou slept so faithful,\nI may contrive our father; and, in their defe"


En este punto del código disponemos de los datos de entrenamiento en el tensor dataseten forma de parejas de secuencias de 100 integers de 32 bits que representan un carácter del vocabulario:

In [11]:
print (dataset)

<MapDataset shapes: ((100,), (100,)), types: (tf.int32, tf.int32)>


En realidad, los datos ya estan preprocesados en el formato que se requiere para ser usados en el entrenamiento de la red neuronal, pero recordemos que en en redes neuronales los datos se agrupan en batches antes de pasarlos al modelo. En nuestro caso hemos decidido un tamaño de batch de 64.

En este codigo, para crear los batches de parejas de secuencias se utiliza tf.data que ademas nos permite barajar las secuencias prebiamente.

In [12]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print (dataset)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>


### Construcción del modelo

Se utilizara una version minima de RNN para simplificar el ejemplo, que contenga solo una capa de LSTM. En concreto se define una red de solo 3 capas:

In [13]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [14]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, 
                      output_dim=embedding_dim, 
                      batch_input_shape=[batch_size, None]))
  model.add(LSTM(rnn_units,
                 return_sequences=True,
                 stateful=True,
                 recurrent_initializer='glorot_uniform'))
  model.add(Dense(vocab_size))
  return model

In [15]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

La primer capa es de tipo Word Embedding, que mapea cada caracter de entrada en un vector Embedding. 

Como argumentos, especificamos el tamaño del vocabulario, especificado con el argumento vocab_size, que indica cuantos vectores Embedding tendra la capa. A continuacion indicamos las dimensiones de estos vectores mediante el argumento embedding_dim. Finalmente se indica el tamaño del batch que usaremos para entrenar.

La segunda capa es de tipo LSTM. En la misma indicamos el numero de neuronas recurrentes con el argumento rnn_units.

Con return_sequence se indica que queremos predecir el caracter siguiente a todos los caracteres de entrada, no solo el siguiente al ultimo caracter.

El argumento stateful indica, explicado de manera simple, el uso de las capacidades de memoria de la red entre baches. Si este argumento esta instanciado a false, indica que con cada nuevo batch se inicializan las memory cells, mientras que si esta en true se esta indicando que para cada batch se mantendran las actualizaciones hechas durante la ejecucion del batch anterior.

El ultimo argumento que usamos es recurrent_kernel, donde indicamos como se deben inicializar los pesos de las matrices internas de la red. Usamos la distibucion glorot_uniform, habitual en estos casos.

Finalmente la ultima capa es de tipo Dense. Aqui lo importante es el argumento units que nos dice cuantas neuronas tendra la capa y que nos marcara la dimension de la salida. En nuestro caso sera igual al tamaño de nuestro vocabulario.

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           15872     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 62)            63550     
Total params: 5,326,398
Trainable params: 5,326,398
Non-trainable params: 0
_________________________________________________________________


Como puede observarse, la capa LSTM consta de una gran cantidad de parametros, mas de 5 millones, como era de esperar. 

Para cada caracter de entrada (transormado a su equivalente numerico), el modelo busco su vector de Embedding correspondiente y luego ejecuta la capa LSTM con este vector como entrada. A la salida de esta capa, aplica la capa Dense para decidir cual es el siguiente caracter.

Inspeccionemos las diemnsiones de los tensores para comprender mas a fondo el modelo. Nos fijamos en el primer batch del conjunto de datos de entrenamiento y observamos su forma:

In [17]:
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")


Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


Vemos que en esta red la secuencia de entrada son de batches de 100 caracteres, pero el modelo una vez entrenado puede ser ejecutado con cualquier tamaño de cadena de entrada.

Como salida el modelo nos devuelve un tensor con una dimension adicional con la verosimilitud para cada caracter del vocabulario:

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):  
    example_batch_predictions = model(input_example_batch)
    print("Prediction: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction:  (64, 100, 62) # (batch_size, sequence_length, vocab_size)


El siguiente paso es elegir uno de los caracteres. Sin entrar en detalle, no se eligira el caracter mas "probable" utilizando por ejemplo argmax, puesto que el modelo puede llegar a entrar en un bucle. Lo que se hara es obtener una muestra de la distribucion de salida. Probandolo para el primer ejemplo en el batch:

In [19]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()

In [20]:
sampled_indices_characters

array([32, 29, 44, 39, 10, 60, 14, 53, 41, 49, 22,  6,  0,  5, 19, 20,  5,
       36,  5, 10, 48, 34, 44, 61, 52,  4, 35, 14, 23, 39, 57, 17, 55, 15,
       14,  5,  9, 15, 41,  3, 53, 21, 53, 31, 59, 12, 61, 56, 39, 17, 29,
        9, 60, 12,  0, 29, 60,  9, 14, 16, 60,  0, 53, 27, 45, 55, 27,  2,
       31, 59, 19, 32,  5, 35, 13, 19, 47, 14,  5, 30, 53, 16,  3, 61, 46,
       23, 18,  9, 37, 61,  6, 12, 33,  2, 27, 32, 23, 17, 53, 16],
      dtype=int64)

# Entrenar el modelo

En este punto, el problema puede tratarse como un problema de clasificacion estandar para el que debemos definir la funcion de costo y el optimizador.

Para la funcion de costo usaremos la funcion estandar categorical_crossentropy dado que estamos considerando datos categoricos. Y dado que el retorno, como hemos visto, se trata de valores de verosimilitud (no de probabilidades como si hubieramos ya aplicado softmax) se instanciara el argumento from_logits a True.

In [21]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

En cuanto al optimizador, usaremos Adam con sus argumentos por defecto.

In [22]:
model.compile(optimizer='adam', loss=loss)

Configurar el *checkpoints*

Esta es una tecnica de tolerancia de fallos para procesos cuyo tiempo de ejecucion es muy largo. La idea es guardar una instancia del estado del sistema periodicamente para recuperar desde ese punto la ejecucion en caso de fallo del sistema. En nuestro caso, cuando entrenamos modelos Deep Learning, el Checkpoint lo forman basicamente los pesos del modelo. Estos Checkpoints se pueden usar tambien para hacer predicciones tal cual como haremos en este ejemplo.

La libreria de Keras proporciona Checkpoints a traves de la API Callbacks. Concretamente usaremos ModelCheckpint para especificar como salvar los Checkpoints a cada epoch durante el entrenamiento, a traves de un argumento en el metodo fit() del modelo.

En el código debemos especificar el directorio en el que se guardarán los Checkpoints que salvaremos y el nombre del fichero (que le añadiremos el número de epoch para nuestra comodidad):

In [23]:
 # directorio
checkpoint_dir = './training_checkpoints'
# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

*Training*

Ahora ya esta todo preparado para empezar a entrenar la red.

In [25]:
EPOCHS=20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
15/15 [==============================] - 57s 4s/step - loss: 2.1178
Epoch 2/20
15/15 [==============================] - 58s 4s/step - loss: 2.0690
Epoch 3/20
15/15 [==============================] - 58s 4s/step - loss: 2.0249
Epoch 4/20
15/15 [==============================] - 59s 4s/step - loss: 1.9799
Epoch 5/20
15/15 [==============================] - 58s 4s/step - loss: 1.9371
Epoch 6/20
15/15 [==============================] - 58s 4s/step - loss: 1.9005
Epoch 7/20
15/15 [==============================] - 58s 4s/step - loss: 1.8595
Epoch 8/20
15/15 [==============================] - 60s 4s/step - loss: 1.8223
Epoch 9/20
15/15 [==============================] - 60s 4s/step - loss: 1.7862
Epoch 10/20
15/15 [==============================] - 60s 4s/step - loss: 1.7471
Epoch 11/20
15/15 [==============================] - 60s 4s/step - loss: 1.7111
Epoch 12/20
15/15 [==============================] - 60s 4s/step - loss: 1.6719
Epoch 13/20
15/15 [==============================

### Generación de texto

Ahora que tenemos el modelo entrenado, podemos pasar a usarlo para generar texto. 

Para realizar este paso de prediccion simple, vamos a usar un tamaño de batcj de 1. Debido a la forma en que se pasa el estado de la RNN de un instante de tiempo al siguiente, el modelo solo acepta un tamaño de batch fijo una vez construido. Por ello, para poder ejecutar el modelo con un tamaño de bartch diferente, necesitamos reconstruir manualmente el modelo con el metodo build() y restaurar sus pesos desde el ultimo checkpoint:

In [26]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_20'

In [27]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

Ahora podemos pasar a generar texto a partir de una palabra de partida con el siguiente codigo:

In [28]:
def generate_text(model, start_string):

  num_generate = 500
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []


  temperature = 0.5

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()


      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

El codigo empieza con inicializaciones como: definir el numero de caracteres a predecir con la variable num_generate, convertir la palabra inicial (start_string) a su correspondiente representacion numerica y preparar los tensores necesarios.

Se usa una variable temperatura para decidir que tan conservador en sus predicciones queremos que se comporte nuestro modelo.

Con "temperaturas altas" (hasta 1) se permitira mas creativivdad al modelo para generar texto, pero a costa de mas errores (por ej, errores ortograficos, etc). Mientras que con "temperaturas bajas" habra menos errores pero el modelo mostrara poca creatividad. 

A partir de este momento empieza el bucle para generar los caracteres que le hemos indicado (que usa el caracter de entrada la primera vez) y luego sus propias predicciones como entrada a cada iteracion al modelo RNN.

Recordemos que estamos en un batch de 1 pero el modelo returno el tensore de batch con las dimensiones que lo habiamos entrenado y por tanto debemos reducir la dimension batch (squeeze).

Luego se usa una distribucion categorica para clacular el indice del caracter predicho.

Este caracter recien predicho, se usa como proxima entrada al modelo, retroalimentando al mismo para que ahora tenga mas contexto (en lugar de solo una letra). Despues de predecir la siguiente letra, se retroalimenta nuevamente, y asi sucesivamente de manera que va aprendiendo a medida que se obtiene mas contexto de los caracteres predichcos previamente.

Ahora que se ha descrito como se ha programado la funcion generate_text() probemos como se comporta el modelo:

In [32]:
print(generate_text(model, start_string=u"a"))

acund father, he hath sonceren shall be stord my partain, and the queen of the raughter of your sendery,
There being come to the stake of your day be the bedices and easter shall not his good bear to every death, but and the stares brow the intament of the gone.

SARANIO:
Here be the cours are here, she is me be stand and acoun a ploine, she dish thing of my bead;
And let de the more that the child and lead not love so fair a hink,
And be my lord I have a true bedore your grace so make my lord;
T


Se pueden probar distintos strings iniciales para observar como se comporta el modelo a cada uno de ellos.

En resumen, el modelo presentado parece que ha aprendido a generar texto de manera interesanto, teniendo en cuenta el reducido dataset inicial con el que se ha entrenado.